In [1]:
!pip install mpu
!pip install geopy
!pip install sentinelhub

In [2]:
import pandas as pd
import numpy as np
import zipfile

In [3]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sentinelhub import SHConfig
import numpy as np
import pandas as pd
import requests
import urllib

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, \
    DataCollection, bbox_to_dimensions, DownloadRequest, to_utm_bbox
from sentinelhub import geo_utils

import sat_utils

## Configuring the API <br>
* Please note that this may not work if done after November 20, which is the date the API subscription is over (after that it costs money to keep up).

In [4]:
#set up the API configuration
CLIENT_ID = '6493e682-3bee-408a-837c-b8e8b2471156'
CLIENT_SECRET = 'd1dU,g8ip8I/n-@.S&5BjW]jZ|YhK+56$v{ymq.P'
config = SHConfig()

if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET
    
if config.sh_client_id == '' or config.sh_client_secret == '':
    print("Warning! Credentials aren't entered")

## Setting the Coordinatess for the boxes to pull satellite images of

In [5]:
OR_coords = [-124.03333, 44.583333, -123.4166667, 44]

## Creating the box objects and the size arrays as needed for requesting the images from the API. I am printing out the sizes to ensure there are less than or equal to 2,500 by 2,500 pixels - this the maximum sized image that can be pulled from the API

In [6]:
print("Oregon 1 Coordinates have size ")
or_bbox, or_size = sat_utils.checkSize(OR_coords,72)
print(or_size)

Oregon 1 Coordinates have size 
(691, 894)


## Since the boxes are sized correctly, I will now request the images through the API

In [7]:
or_image = sat_utils.getSatelliteImage(or_bbox, or_size,'Oregon1',['07/01/2020','07/15/2020'],'leastCC',config)

## Getting All Bands Available through the API

In [8]:
or_2_bands = sat_utils.getSecondFourBands(or_bbox, or_size,'Oregon',['07/01/2020','07/15/2020'],'leastCC',config)
or_3_bands = sat_utils.getThirdFourBands(or_bbox, or_size,'Oregon',['07/01/2020','07/15/2020'],'leastCC',config)
or_fl_bands = sat_utils.getFirstAndLastBands(or_bbox, or_size,'Oregon',['07/01/2020','07/15/2020'],'leastCC',config)
or_all_bands = sat_utils.combine([or_image[0],or_2_bands[0],or_3_bands[0],or_fl_bands[0]])

## Getting the Elevation Data

In [9]:
or_elevations = sat_utils.getSentinelElevation(or_bbox, or_size, ['07/01/2020','07/31/2020'],config)

## Calculating the Coordinates for each Pixel

In [10]:
or_coordinates = sat_utils.getCoordinates(OR_coords,or_bbox,or_image)

0 of 894
100 of 894
200 of 894
300 of 894
400 of 894
500 of 894
600 of 894
700 of 894
800 of 894


## Creating the final compacted data sets

Below I have converted the compacted image and elevation data into a dataframe of size (image width * image length) x #features rather than having an array of size image width x image length x #features.

In [11]:
final_compacted_data_or = sat_utils.finalizeData(or_all_bands, or_elevations[0], or_coordinates)

In [12]:
final_compacted_data_or.columns = ['B04','B03','B02','B05','B06','B07','B08','B8A','B09','B10','B11','B01','B12','Elevation','Lat','Lon','i','j']

## Feature Engineering

Explanations of the different bands (B02 through B12) as well as further explanations of the indices added below can be found [here](https://custom-scripts.sentinel-hub.com/) under 'Sentinel 2'.

<b>Moisture Index (MSI)</b> - This is B11/B08. This measures the water content in the canopy leaves of a forest. The value ranges from 0 to over 3, but the values for green vegetation are usually between 0.4 and 2.

In [13]:
final_compacted_data_or['MSI'] = final_compacted_data_or['B11']/final_compacted_data_or['B08']

<b>Chlorophyl Red Edge (CRE)</b> - This is B07/B05 - 1.

In [14]:
final_compacted_data_or['CRE'] = final_compacted_data_or['B07']/final_compacted_data_or['B05'] - 1

<b>Enhanced Vegetation Index (EVI) </b>- This corrects for soil background signals and atmospheric influences. It's good in areas of dense canopy coverage. The ranges of values are between -1 and 1. Healthy vegetation ranges from 0.2 to 0.8.

In [15]:
or_evi, or_end_EVI = sat_utils.getEVI(final_compacted_data_or['B08'],final_compacted_data_or['B04'],final_compacted_data_or['B02'])
final_compacted_data_or['EVI'] = or_evi
final_compacted_data_or[['EVI1','EVI2','EVI3']] = or_end_EVI

<b>Normalized Difference Vegetation Index (NDVI)</b> - This normalizes green leaf scattering in Near Infra-red wavelengths with chlorophyll absorption in red wavelengths.

In [16]:
or_NDVI, or_end_NDVI = sat_utils.getNDVI(final_compacted_data_or['B08'],final_compacted_data_or['B04'])
final_compacted_data_or['NDVI'] = or_NDVI
final_compacted_data_or[['NDVI1','NDVI2','NDVI3']] = or_end_NDVI

<b>Green Normalized Difference Vegetation Index (GNDVI)</b> - This is similar to NDVI but is more sensitive to variation in chlorophyll content.

In [17]:
or_gndvi = (final_compacted_data_or['B08']-final_compacted_data_or['B03'])/(final_compacted_data_or['B08']+final_compacted_data_or['B03'])
final_compacted_data_or['GNDVI'] = or_gndvi

<b>Modified Chlorophyll Absorption in Reflectance Index (MCARI)</b> - This gives a measure of the depth of chlorophyll absorption.

In [18]:
or_mcari = ((final_compacted_data_or['B05']-final_compacted_data_or['B04'])-0.2*(final_compacted_data_or['B05']-final_compacted_data_or['B03']))*(final_compacted_data_or['B05']/final_compacted_data_or['B04'])
final_compacted_data_or['MCARI'] = or_mcari

<b>Leaf Area Index (LAI)</b> - This measures the green-leaf area in a unit of land.

In [19]:
or_lai = sat_utils.getLAI(or_bbox, or_size, 'oregon_LAI',['07/01/2020','07/15/2020'],'leastCC',config)
final_compacted_data_or['LAI'] = sat_utils.convertToSingle(or_lai[0])

<b>Color Correction (CC)</b> - This is a Sentinel Hub algorithm based transformation of the red, green, and blue values. It performs a linear atmospheric correction and applies a curve to the color components. This enhances details in the dark areas. Since most the areas in these satellite images are dark green, this may help distinguish the different shades.

In [20]:
or_cc = sat_utils.colorCorrect(or_bbox, or_size, 'oregon_cc',['07/01/2020','07/15/2020'],'leastCC',config)
final_compacted_data_or[['CC Red','CC Green','CC Blue']] = sat_utils.convertToSingle(or_cc[0])

<b>Plant Senescence Reflectance Index (PSRI)</b> - This index is used to monitor the health of vegetation.

In [21]:
or_PSRI = sat_utils.getPSRI(or_bbox, or_size, 'oregon_cc',['07/01/2020','07/15/2020'],'leastCC',config)
final_compacted_data_or['PSRI'] = sat_utils.convertToSingle(or_PSRI[0])

## Writing the Final Data Set to a CSV File

In [23]:
final_compacted_data_or.loc[:300000,:].to_csv('Data/Compacted Data Set1.csv',index=False)
final_compacted_data_or.loc[300001:,:].to_csv('Data/Compacted Data Set2.csv',index=False)

In [24]:
zipfile.ZipFile('Data/Compacted Data Set1.zip','w',zipfile.ZIP_DEFLATED).write('Data/Compacted Data Set1.csv')
zipfile.ZipFile('Data/Compacted Data Set2.zip','w',zipfile.ZIP_DEFLATED).write('Data/Compacted Data Set2.csv')

<b>Sources:</b> <br>
I used this [source](https://custom-scripts.sentinel-hub.com/) for many of my scripts for interfacing with the Sentinel Hub Python API.<br>
I used Python code from this [source](https://sentinelhub-py.readthedocs.io/en/latest/examples/processing_api_request.html) for integrating scripts from the above source into the Sentinel Hub Python API.